## 🧭 Argo FormatChecker Notebook (AMRIT Consortium)

This notebook allows you to use the **Argo Format Checker** provided by  **AMRIT** to validate Argo NetCDF files.  
The FormatChecker performs both **format** and **content** checks on Argo NetCDF files to ensure compliance with the Argo data standards.

📘 **References:**
- The Argo NetCDF format is defined in the [Argo User’s Manual](http://dx.doi.org/10.13155/29825).  
- More details and documentation are available on the [Argo Data Management website](https://www.argodatamgt.org/Documentation).


## The main steps to run the checker

1. Setup 
    - before running this notebook, install dependencies with:
    ```bash
    pip install -r notebooks/requirements.txt
    ```
2. Run the complete notebook
    - Imports necessary packages.
    - Configures the API URL and DAC.
        - Set the API_BASE_URL to where the File Checker API is running.
        - If you are running the File Checker locally inside Docker, set it to the address where the 
            Docker container is exposing the API `http://localhost:8000`
        - If you are running the File Checker in a Kubernetes cluster, use the cluster
            URL where the service is exposed. For example:
            `https://livkrakentst.clusters.bodc.me/ewetchy/amrit/argo-toolbox/api/file-checker`
        - Only one of the URLs should be uncommented based on your environment.
    - Sets the default DAC for validation.
    - Checks the connectivity to the API.
5. Checking files
    - <p>If you need to check few selected files , select them by clicking on the <b>Upload</b> button.</p>
6. Optionally run the checker on all files of a deployment.
    - <p>If you want to check a whole deployment, select the path of the files and select <b> Run Deployment Check </b>.</p>
7. Results are saved to a csv file.Result of first few are shown on the console.
    - The name of the output file is part of configuration

In [ ]:
# ===============================
# 1️⃣ Importing necessary packages
# ===============================

import logging
import tempfile
import time
from pathlib import Path
from typing import Any

import ipywidgets as widgets
import pandas as pd
import requests
from IPython.display import display as ipy_display
from ipywidgets import FileUpload, Label, Output, VBox

logging.basicConfig(level=logging.INFO, format="%(levelname)s: %(message)s")
logger = logging.getLogger(__name__)
for handler in logger.handlers[:]:
    logger.removeHandler(handler)
logger.setLevel(logging.INFO)


# ===============================
# 2️⃣ Configuration
# ===============================
# Local Docker instance
API_BASE_URL="http://localhost:8000"

# Kubernetes test instance
# API_BASE_URL= "https://livkrakentst.clusters.bodc.me/ewetchy/amrit/argo-toolbox/api/file-checker"

# Default DAC for validation
DEFAULT_DAC = "bodc"

# Mount location of all the deployment files
# if you are running the API locally via Docker, ensure this path matches the volume mount in your Docker setup
# for example docker  run --rm --name argo-file-checker2 -p 8000:8000 argo-file-checker

# Result file for full deployment checks
deployments_files_check_result_file="results/deployment_files_check_result.csv"

# Endpoints
CHECK_FILE_ENDPOINT = "/check-files"
FULL_DEPLOYMENT_CHECK_ENDPOINT = "/check-deployment"
HEALTH_ENDPOINT = "/"

# Request settings
TIMEOUT = 30  # API request timeout in seconds
HEADERS = {
    "accept": "application/json",
}

# Full URL for file check endpoint
FILE_CHECK_URL = f"{API_BASE_URL}/{CHECK_FILE_ENDPOINT}"

logger.info("🔗 API Base URL:%s",  API_BASE_URL)
logger.info("🏛️ DAC: %s" ,DEFAULT_DAC)
# ============================================
#  3️⃣ function to test config and api
# ============================================
def test_api_connection() -> bool:
    """Test if the API is accessible."""
    logger.info("\n🔍 Testing API Connection..")

    try:
        response = requests.get(f"{API_BASE_URL}/", timeout=5)
        response.raise_for_status()

    except requests.exceptions.ConnectionError:
        logger.exception("Could not connect to API. Is the container running?")
        return False

    except requests.exceptions.Timeout:
        logger.exception("Connection timed out")
        return False

    except Exception:
        logger.exception("Unexpected error")
        return False
    else:
        result = response.json()
        logger.info("API is accessible!")
        logger.info("Health Check Response: %s", result)
        return True

# ============================================
#  4️⃣ function to check the files
# ============================================
def prepare_files(file_paths:list[str])-> list[tuple[str, tuple[str, Any, str]]]:
    """Filechecker to check nc files."""
    files_data = []
    for file_path in file_paths:
        path = Path(file_path)
        if not path.exists():
            logger.warning("File not found: %s",file_path)
            continue

        # Determine content type based on file extension
        content_type = {
            ".json": "application/json",
            ".nc": "application/x-netcdf",
        }.get(path.suffix, "application/x-netcdf")

        try:
            file_obj = path.open("rb")
            files_data.append(("files", (path.name, file_obj, content_type)))
        except Exception:
            logger.exception("Error opening  %s",path)
            continue
    return files_data

def send_file_check(files_data: list[tuple], dac: str) -> dict[str, Any] | None:
    """Send files to FILE_CHECK_URL and return structured result."""
    if not files_data:
        logger.warning("No valid files to upload!")

    params = {"dac": dac}
    start_time = time.time()

    try:
        response = requests.post(
            FILE_CHECK_URL,
            files=files_data,
            params=params,
            headers=HEADERS,
            timeout=TIMEOUT,
        )
    except requests.exceptions.Timeout:
        logger.exception("Request timed out after %s seconds", TIMEOUT )

    except requests.exceptions.RequestException:
        logger.exception("Request error")

    except Exception:
        logger.exception("Unexpected error")

    else:
        end_time = time.time()
        processing_time = end_time - start_time


        for _, (_, file_obj, _) in files_data:
            file_obj.close()
        try:
            result = response.json()
        except ValueError:
            result = response.text

        success = True

        return {
            "success": success,
            "status_code": response.status_code,
            "processing_time": processing_time,
            "result": result,
        }
    finally:
        # This block runs no matter what, ensuring file handles are closed
        for _, (_, file_obj, _) in files_data:
                file_obj.close()

# ============================================
#  5️⃣ function to check the whole deployment
# ============================================
# A mount was created for a whole deployment and that folder is passed.
def file_check_deployment(deployment_files_folder:str, dac:str=DEFAULT_DAC)-> dict[str, Any] | None:
    """Filechecker to check nc files for a whole deployment."""
    logger.info("\n🔍 Checking full deployment in folder: %s", deployment_files_folder)
    # Check that folder exists
    if not Path(deployment_files_folder).exists() or not Path(deployment_files_folder).is_dir():
        logger.error("Deployment folder does not exist: %s", deployment_files_folder)

    # Count number of files in the folder
    file_list = [str(f) for f in Path(deployment_files_folder).iterdir() if f.is_file()]
    num_files = len(file_list)
    logger.info("Number of files found in deployment folder: %d", num_files)

    if num_files == 0:
        logger.warning("No files found in deployment folder: %s", deployment_files_folder)

    files_data = prepare_files(file_list)
    return send_file_check(files_data, dac)

# ==========================================
#  5️⃣ function to display the results and save to CSV
# ============================================
def show_result(result: dict, save_path: str | None = None) -> None:
    """Nicely display the result dict in Jupyter."""
    if not isinstance(result, dict):
        logger.info(result)
        return

    results = result.get("result", {}).get("results", [])
    if not results:
        logger.info("No results found.")
        return

    for r in results:
        r["errors_messages"] = "\n".join(r.get("errors_messages", []))
        r["warnings_messages"] = "\n".join(r.get("warnings_messages", []))
    df = pd.DataFrame(results, columns=[
        "file",
        "result",
        "phase",
        "errors_number",
        "warnings_number",
        "errors_messages",
        "warnings_messages",
    ])

    # Optional: Save to CSV
    if save_path:
        df.to_csv(save_path, index=False, encoding="utf-8")
        logger.info("✅ Results saved to %s",save_path)
    logger.info("✅ Results of first few files %s",save_path)
    ipy_display(df[:3])

# Test 1: Check API connection
api_available = test_api_connection()

if not api_available:
    logger.info("\n API is not accessible. Please check:")
else:
    logger.info("\n🎉 API is ready for use!")
# 2. Upload file(s) to test
label = Label("(for checking few files) Please upload files to check:")
upload = FileUpload(accept=".json,.nc", multiple=True)
out = Output()

def on_upload_change(_change: dict[str, object]) -> None:
    """To handle file upload and trigger file check."""
    with out:
        out.clear_output()
        file_paths = []
        with tempfile.TemporaryDirectory() as tmp_dir:
            for fileinfo in upload.value:
                fname = fileinfo["name"]

                tmp_path = Path(tmp_dir) / fname
                with Path(tmp_path).open("wb") as f:
                    f.write(fileinfo["content"])
                file_paths.append(tmp_path)


            files_data = prepare_files(file_paths)
            res= send_file_check(files_data, DEFAULT_DAC)
            if res is not None:
                show_result(res, save_path="file_check_results.csv")
            else:
                logger.error("File check failed, nothing to display")

upload.observe(on_upload_change, names="value")
ipy_display(VBox([label, upload, out]))
# Create a text input widget for the deployment folder

deployment_folder_widget = widgets.Text(
        value="//wsl.localhost/Ubuntu/home/vidkri/argo_mount",  # default value
        placeholder="Enter path to deployment folder",
        description="Enter path to deployment folder:",
        layout=widgets.Layout(width="90%"),
        style={"description_width": "200px"},
    )

# Create a button to trigger the check

run_button = widgets.Button(description="Run complete deployment Check", button_style="success")

# Define button click handler
def on_run_button_clicked(_b:dict[str, object]) -> None:
    """Handle deployment check on button click."""
    with output:
        output.clear_output(wait=True)
        deployment_folder = deployment_folder_widget.value.strip()
        if not Path(deployment_folder).exists():
            logger.warning("Folder does not exist: %s" , deployment_folder)
            return
        # Run your deployment check
        start = time.perf_counter()
        result = file_check_deployment(deployment_folder, DEFAULT_DAC)
        end = time.perf_counter()
        elapsed_time = end - start

        logger.info("Full deployment check result is saved to a file :%s", deployments_files_check_result_file)
        if result is not None:
            logger.info("Checking full deployment took %s seconds", elapsed_time)
            show_result(result, deployments_files_check_result_file)
        else:
            logger.error("Deployment file check for the deployment failed")
# Output area
output = widgets.Output()


# Link button to handler
run_button.on_click(on_run_button_clicked)

# Display widgets
display(deployment_folder_widget, run_button, output)